# Retrieve DNS queries from zeek logs

In [2]:
import pandas as pd
# import numpy as np 
# import binascii # Binary to Ascii 
# from pathlib import Path
# from datetime import datetime
import pickle
# import io
import glob
# from tqdm import tqdm
# from datetime import datetime, timedelta

In [3]:
fields = ['ts',	'uid',	'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',	'proto', 'trans_id', 'rtt',	'query', 'qclass', 'qclass_name', 'qtype', 'qtype_name', 'rcode']

In [4]:
def zeekToCsv(conlog: str, fieldz: list):
    outcsv = 'dnstemp.csv'
    with open(conlog) as file:
        lines = file.read().splitlines()
        with open(outcsv,"w") as outfile:
            for line in lines:
                if line[0] == '#':
                    continue
                l = (line.split())[0:10]
                tmp = ",".join(l)
                outfile.write(tmp + '\n')

In [5]:
logs = glob.glob('zeeklogs/*/dns.log')
dnslogs = pd.DataFrame()
for log in logs:
    print(log)
    zeekToCsv(log, fields)
    dnslogs = pd.concat([dnslogs, pd.read_csv("dnstemp.csv", names=fields)])  

zeeklogs/aug3/dns.log
zeeklogs/jul14/dns.log
zeeklogs/jul25/dns.log
zeeklogs/aug6/dns.log
zeeklogs/aug1/dns.log
zeeklogs/aug9/dns.log
zeeklogs/jul28/dns.log
zeeklogs/jul26/dns.log
zeeklogs/jul27/dns.log
zeeklogs/jul18/dns.log


In [6]:
dnslogs

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,trans_id,rtt,query,qclass,qclass_name,qtype,qtype_name,rcode
0,1.659544e+09,CQkTmN3VXfTDBpuA6,192.168.70.4,62926,8.8.8.8,53,udp,43941,0.032229,onedscolprdeus00.eastus.cloudapp.azure.com,NaN,NaN,NaN,NaN,NaN
1,1.659544e+09,CAly4B2czC4rkplNL2,192.168.70.160,60435,192.168.70.4,53,udp,3052,0.034029,v10.events.data.microsoft.com,NaN,NaN,NaN,NaN,NaN
2,1.659544e+09,CtkpHJ2jj5zGF4ZX41,192.168.70.160,60435,8.8.8.8,53,udp,3052,0.025886,v10.events.data.microsoft.com,NaN,NaN,NaN,NaN,NaN
3,1.659544e+09,Csrq1B3gxaWYyLhVra,fe80::b159:f72e:fb02:fcb9,5353,ff02::fb,5353,udp,0,-,_nmea-0183._tcp.local,NaN,NaN,NaN,NaN,NaN
4,1.659544e+09,CXNzoW1bYOuaVHMHF5,192.168.70.101,5353,224.0.0.251,5353,udp,0,-,_nmea-0183._tcp.local,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8772,1.658190e+09,CFsM97VJ9G61wXmlc,192.168.70.4,59054,8.8.8.8,53,udp,38910,-,presence.teams.live.com,NaN,NaN,NaN,NaN,NaN
8773,1.658190e+09,CkYKfc407nMXma8Ie1,192.168.70.4,59177,8.8.8.8,53,udp,18433,-,e10370.g.akamaiedge.net,NaN,NaN,NaN,NaN,NaN
8774,1.658190e+09,C6U2u01aeJ4mLRoOAd,192.168.70.4,58923,8.8.8.8,53,udp,56593,-,e12358.g.akamaiedge.net,NaN,NaN,NaN,NaN,NaN
8775,1.658190e+09,CQTzUv36VTEUBQKl0e,192.168.70.4,59040,8.8.8.8,53,udp,8089,-,www.google.com,NaN,NaN,NaN,NaN,NaN


In [6]:
contain_values = dnslogs[dnslogs['query'].str.contains('nowstudios.xyz')]
x = contain_values['uid'].tolist()

In [7]:
pickle.dump(x, open('dnstunneluids.pickle', 'wb'))